# Importing the libraries

In [1]:
import numpy as np
import random
import os
import tensorflow as tf
from tensorflow import keras
import platform
from tensorflow.keras import layers, optimizers, losses, models, Input, Model
import time # Per misurare il tempo di training
from tensorflow.keras.callbacks import EarlyStopping # Per l'early stopping
import matplotlib.pyplot as plt
from tqdm import tqdm # Per mostrare una barra di progresso

# Set seeds for random operations.

In [2]:
# --- 1. Impostazione del Seed Globale all'inizio del tuo script ---
# Questo è il punto chiave per la riproducibilità di TUTTO ciò che segue.
MASTER_RANDOM_SEED = 42
np.random.seed(MASTER_RANDOM_SEED)
random.seed(MASTER_RANDOM_SEED) # Imposta anche il seed per la libreria 'random' di Python se la usi
tf.random.set_seed(MASTER_RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(MASTER_RANDOM_SEED) # Per operazioni basate su hash (es. ordine dei dizionari)
os.environ['TF_DETERMINISTIC_OPS'] = '1' # Forza operazioni deterministiche in TensorFlow 2.x

# Print the HW Specs.

In [3]:
print("--- Dettagli dell'Architettura Hardware della Sessione Colab ---\n")

# --- 1. Dettagli CPU ---
print("--- Dettagli CPU ---")
!lscpu
print("\n")

# --- 2. Dettagli RAM (Memoria) ---
print("--- Dettagli RAM (Memoria) ---")
!cat /proc/meminfo | grep MemTotal
print("\n")

# --- 3. Dettagli Spazio su Disco ---
print("--- Dettagli Spazio su Disco ---")
!df -h /
print("\n")

# --- 4. Dettagli Acceleratore Hardware (GPU/TPU) ---
print("--- Dettagli Acceleratore Hardware (GPU/TPU) ---")
try:
    tpu_address = os.environ.get('COLAB_TPU_ADDR')
    if tpu_address:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print(f"Tipo Acceleratore: TPU (indirizzo: {tpu_address})")
        print("Dispositivi TPU disponibili:")
        for device in tf.config.list_logical_devices('TPU'):
            print(f"  - {device.name}")
    else:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            print(f"Tipo Acceleratore: GPU")
            for gpu in gpus:
                print(f"  - Dispositivo GPU rilevato: {gpu.name}")
            print("\nDettagli GPU specifici (da `!nvidia-smi`):")
            !nvidia-smi
        else:
            print("Tipo Acceleratore: Nessuna GPU o TPU rilevata (in uso CPU)")

except Exception as e:
    print(f"Si è verificato un errore durante la rilevazione dell'acceleratore: {e}")
    print("Tentativo di rilevare i dispositivi TensorFlow standard:")
    devices = tf.config.list_logical_devices()
    if devices:
        for device in devices:
            print(f"  - Dispositivo rilevato: {device.name}, Tipo: {device.device_type}")
    else:
        print("Nessun dispositivo TensorFlow rilevato.")

print("\n--- Analisi Dettagli Hardware Completata ---")

--- Dettagli dell'Architettura Hardware della Sessione Colab ---

--- Dettagli CPU ---
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4

# Connect To Gdrive to store the datasets created.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Define the paths

In [5]:
# Percorsi dataset
paths = {
    "0-20": {
        "train": "/content/drive/MyDrive/GitHub/AWGN/dataset/training_0-20_SNR.npz",
        "val": "/content/drive/MyDrive/GitHub/AWGN/dataset/validation_0-20_SNR.npz",
    },
    "11-15": {
        "train": "/content/drive/MyDrive/GitHub/AWGN/dataset/training_11-15_SNR.npz",
        "val": "/content/drive/MyDrive/GitHub/AWGN/dataset/validation_11-15_SNR.npz",
    }
}

# Directory salvataggio modelli
save_dir = "/content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks"
os.makedirs(save_dir, exist_ok=True)

# Load the datasets.

In [6]:
def load_dataset(filepath):
    """
    Carica i dati X e y da un file .npz.
    Si aspetta che il file contenga 'X_train' o 'X_val' e 'y_train' o 'y_val'.
    """
    data = np.load(filepath)

    # Controlla se le chiavi per il training sono presenti
    if 'X_train' in data and 'y_train' in data:
        print(f"  Caricato Training data da: {filepath}")
        return data['X_train'], data['y_train']
    # Altrimenti, controlla se le chiavi per la validation sono presenti
    elif 'X_val' in data and 'y_val' in data:
        print(f"  Caricato Validation data da: {filepath}")
        return data['X_val'], data['y_val']
    else:
        # Se nessuna delle combinazioni attese è trovata, solleva un errore
        raise ValueError(f"Il file {filepath} non contiene i dati X e y attesi (né 'X_train'/'y_train' né 'X_val'/'y_val'). "
                         f"Chiavi trovate: {list(data.keys())}")

# Benchmark models definition

In [7]:
# Funzioni modello

def get_ultracan_model(input_shape, model_name='ULTRA-CAN'):
    inputs = Input(shape=input_shape)

    # Reshape per conv1D
    x = layers.Reshape((input_shape[0], 1), name="reshape")(inputs)

    # Leggera CNN
    x = layers.Conv1D(8, kernel_size=3, padding='same', activation='relu', name="cnn1")(x)
    x = layers.Conv1D(8, kernel_size=3, padding='same', activation='relu', name="cnn2")(x)

    # Temporal convolution (tipo TCN semplificato con dilatazione)
    x = layers.Conv1D(8, kernel_size=3, dilation_rate=2, padding='same', activation='relu', name="tcn1")(x)

    # Attention leggera (tipo global self-attention)
    attention = layers.Permute((2,1), name="permute_for_attention")(x)
    attention = layers.Dense(input_shape[0], activation='softmax', name="attention_weights")(attention)
    attention = layers.Permute((2,1), name="unpermute")(attention)
    x = layers.Multiply(name="apply_attention")([x, attention])

    # Pooling e FC finale
    x = layers.GlobalAveragePooling1D(name="global_avg_pool")(x)
    x = layers.Dense(32, activation='relu', name="fc1")(x)
    outputs = layers.Dense(2, activation='softmax', name="output")(x)

    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

def get_tdnn_model(input_shape, model_name='TDNN OFDM-DCSK'):
    # Dal paper: "Reliable and Secure Deep Learning-Based OFDM-DCSK Transceiver Design Without Delivery of Reference Chaotic Sequences"
    inputs = Input(shape=input_shape)
    input_length = input_shape[0]
    x = layers.Reshape((input_length, 1), name="reshape_for_conv")(inputs)
    x = layers.Conv1D(4, 2, activation='relu', padding='same', name="tdnn_conv1")(x)
    x = layers.Conv1D(8, 4, activation='relu', padding='same', name="tdnn_conv2")(x)
    x = layers.Conv1D(16, 8, activation='relu', padding='same', name="tdnn_conv3")(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=False), name="bidirectional_lstm")(x)
    x = layers.Dense(128, name="fc1")(x)
    x = layers.BatchNormalization(name="batch_norm")(x)
    x = layers.Activation('relu', name="relu_fc1")(x)
    outputs = layers.Dense(2, activation='softmax', name="output_softmax")(x)
    model = Model(inputs, outputs, name='DNN_Aided_Demodulator')
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def get_mc_dlcs_model(input_shape, model_name='MC-DLCSK'):
    # Dal paper: "A Multi-Carrier Deep Learning CSK Communication System"
    if input_shape[0] % 2 != 0:
        raise ValueError("input_shape deve essere pari per MC-DLCSK")
    inputs = Input(shape=input_shape)
    x = layers.Reshape((input_shape[0], 1), name="reshape_for_lstm")(inputs)
    x = layers.Bidirectional(layers.LSTM(50, return_sequences=False), name="bidirectional_lstm")(x)
    x = layers.Dense(64, activation='relu', name="fc1_relu")(x)
    outputs = layers.Dense(2, activation='softmax', name="output_softmax")(x)
    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def get_lstm_dnn_model(input_shape, model_name='LSTM_DNN_OFDM_DCSK'):
    # Dal paper: "Intelligent and Reliable Deep Learning LSTM Neural Networks-Based OFDM-DCSK Demodulation Design"
    inputs = Input(shape=input_shape)
    if len(input_shape) == 1:
        x = layers.Reshape((1, input_shape[0]))(inputs)
    elif len(input_shape) == 2:
        x = inputs
    else:
        raise ValueError("input_shape deve essere 1D o 2D")
    x = layers.LSTM(19, return_sequences=True)(x)
    x = layers.TimeDistributed(layers.Dense(38, activation='relu'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization())(x)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(2, activation='softmax')(x)
    model = Model(inputs, outputs, name=model_name)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

models_dict = {
    "Ultra-CAN": get_ultracan_model,
    #"TDNN OFDM-DCSK": get_tdnn_model,
    #"MC-DLCSK": get_mc_dlcs_model,
    #"LSTM_DNN_OFDM_DCSK": get_lstm_dnn_model
}

# Training...

In [8]:
# Addestramento
for snr_range, datasets in paths.items():
    x_train, y_train = load_dataset(datasets['train'])
    x_val, y_val = load_dataset(datasets['val'])

    for model_name, builder in models_dict.items():
        print(f"\nAddestramento modello: {model_name} | SNR range: {snr_range}")
        model = builder(input_shape=x_train.shape[1:])

        callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        start_time = time.time() # Registra il tempo di inizio training

        model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val),
            epochs=30,
            batch_size=32,
            callbacks=[callback],
            verbose=1
        )

        end_time = time.time() # Registra il tempo di fine training
        total_training_time = end_time - start_time

        print(f"Training completato.")
        print(f"Tempo totale di training: {total_training_time:.2f} secondi.") # Stampa il tempo totale

        save_path = os.path.join(save_dir, f"{model_name.replace(' ', '_')}_awgn_snr_{snr_range}.h5")
        model.save(save_path)
        print(f"Salvato modello in: {save_path}")

  Caricato Training data da: /content/drive/MyDrive/GitHub/AWGN/dataset/training_0-20_SNR.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/AWGN/dataset/validation_0-20_SNR.npz

Addestramento modello: Ultra-CAN | SNR range: 0-20
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - accuracy: 0.8331 - loss: 0.2962 - val_accuracy: 0.9998 - val_loss: 9.6936e-04
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - accuracy: 0.9998 - loss: 0.0012 - val_accuracy: 0.9998 - val_loss: 0.0013
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9998 - loss: 8.6382e-04 - val_accuracy: 0.9961 - val_loss: 0.0101
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9991 - loss: 0.0027 - val_accuracy: 0.9994 - val_loss: 0.0018
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9993 - loss: 0.0028 - val_accuracy: 0.9991 - val_loss: 0.0029
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - accuracy: 0.9990 - loss: 0.0026 - va

Training completato.
Tempo totale di training: 200.40 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/Ultra-CAN_awgn_snr_0-20.h5
  Caricato Training data da: /content/drive/MyDrive/GitHub/AWGN/dataset/training_11-15_SNR.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/AWGN/dataset/validation_11-15_SNR.npz

Addestramento modello: Ultra-CAN | SNR range: 11-15
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.8672 - loss: 0.2625 - val_accuracy: 0.9874 - val_loss: 0.0272
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9996 - loss: 0.0011 - val_accuracy: 0.9997 - val_loss: 0.0012
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9998 - loss: 7.0956e-04 - val_accuracy: 1.0000 - val_loss: 4.7074e-05
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 1.0000 - loss: 8.4004e-05 - val_accuracy: 1.0000 - val_loss: 1.3232e-05
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 

Training completato.
Tempo totale di training: 267.06 secondi.
Salvato modello in: /content/drive/MyDrive/GitHub/AWGN/trained_models/benchmarks/Ultra-CAN_awgn_snr_11-15.h5
